In [113]:
import os
#os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import sys
import bwapiEnums


In [114]:
inputUnitCount = 256
outputUnitCount = 256
output_sequence_length = 5
unitTypeTextVectorizationLayer = keras.layers.TextVectorization(output_sequence_length=output_sequence_length, vocabulary=bwapiEnums.allUnits)
unitActionsTextVectorizationLayer = keras.layers.TextVectorization(output_sequence_length=output_sequence_length, vocabulary=bwapiEnums.unitActions)
unitOrdersTextVectorizationLayer = keras.layers.TextVectorization(output_sequence_length=output_sequence_length, vocabulary=bwapiEnums.unitOrders)

In [115]:
# Input layers
unitTypeInput = layers.Input(shape=(inputUnitCount,), dtype=tf.string, name='UnitTypeInput')
unitOrderInput = layers.Input(shape=(inputUnitCount,), dtype=tf.string, name='UnitOrderInput')
positionInput = layers.Input(shape=(inputUnitCount, 2), dtype=tf.float32, name='PositionInput')

In [116]:
#unitTypeVectorized = layers.Lambda(lambda x: unitTypeTextVectorizationLayer(x))(unitTypeInput)
#unitTypeVectorized = unitTypeTextVectorizationLayer(unitTypeInput)
# Vectorize the unit type and unit order inputs
unitTypeVectorized_list = [unitTypeTextVectorizationLayer(unitTypeInput[:, i]) for i in range(inputUnitCount)]
unitOrderVectorized_list = [unitOrdersTextVectorizationLayer(unitOrderInput[:, i]) for i in range(inputUnitCount)]

# Stack the vectorized outputs back to rank-2 tensors
unitTypeVectorized = layers.Concatenate(axis=1)(unitTypeVectorized_list)
unitOrderVectorized = layers.Concatenate(axis=1)(unitOrderVectorized_list)

# Reshape the stacked outputs
unitTypeVectorized = layers.Reshape((inputUnitCount, -1))(unitTypeVectorized)
unitOrderVectorized = layers.Reshape((inputUnitCount, -1))(unitOrderVectorized)

# Convert the vectorized outputs to float32
unitTypeVectorized = tf.cast(unitTypeVectorized, tf.float32)
unitOrderVectorized = tf.cast(unitOrderVectorized, tf.float32)

In [117]:

# Combine the vectorized inputs and position input
combinedInputs = layers.Concatenate(axis=-1)([unitTypeVectorized, unitOrderVectorized, positionInput])

In [118]:
# Add more layers as needed for your specific use case
hiddenLayer = layers.Dense(inputUnitCount*10, activation='relu')(combinedInputs)

#output_shape = (inputUnitCount, len(bwapiEnums.unitActions))
unitActionOutputLayer = layers.Dense(len(bwapiEnums.unitActions), activation='softmax')(hiddenLayer)
#unitActionOutputLayer = layers.Reshape(target_shape=output_shape)(unitActionOutputLayer)

unitOrderPositionOutputLayer = layers.Dense(2, activation='relu')(hiddenLayer)
model = keras.models.Model(inputs=[unitTypeInput, unitOrderInput, positionInput], outputs=[unitActionOutputLayer,unitOrderPositionOutputLayer])


In [119]:
#model.summary()

In [120]:
position = np.random.rand(256, 2)
unitTypes = np.random.choice(bwapiEnums.allUnits,size=256)
unitOrders = np.random.choice(bwapiEnums.unitOrders,256)

# Combine the input data into a single array with multiple dimensions
inputData = [np.array([unitTypes]), np.array([unitOrders]), np.array([position])]

In [121]:
# Make a prediction using the model
prediction = model.predict(inputData)

1/1 [==============================] - 13s 13s/step


In [122]:
#print(prediction[0])
print(prediction[0][0][0])
predictedActions = []
for i in range(inputUnitCount):
    seq = prediction[0][:, i] # Get the predicted sequence for the i-th input unit
    idx = np.argmax(seq) # Get the index of the highest predicted probability for the sequence
    action = unitActionsTextVectorizationLayer.get_vocabulary()[idx] # Look up the corresponding string in the vocabulary
    predictedActions.append(action)

# Print the predicted actions
print(predictedActions)

[0.14660926 0.1344882  0.14023484 0.15078779 0.14203045 0.13725783
 0.14859161]
['patrol', 'patrol', 'patrol', 'train', 'patrol', 'train', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'train', 'train', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'train', 'patrol', 'patrol', 'patrol', 'train', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'train', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'train', 'patrol', 'train', 'patrol', 'patrol', 'patrol', 'patrol', 'train', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'train', 'patrol', 'patrol', 'train', 'patrol', 'patrol', 'train', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'patrol', 'train', 'patrol', 'patrol', 'patrol', 'train', 'patrol', 'train', 'patr

In [125]:
print(np.min(prediction[0]), np.max(prediction[0]))
print(np.min(prediction[1]), np.max(prediction[1]))

0.13441335 0.15241541
0.0 0.014810957
0.0006090391
